In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prodzar/stocks-historical-price-data")

print("C:/Users/supre/Documents/FAI/PROJECT", path)

100%|██████████| 191k/191k [00:00<?, ?B/s]

Extracting files...
C:/Users/supre/Documents/FAI/PROJECT C:\Users\supre\.cache\kagglehub\datasets\prodzar\stocks-historical-price-data\versions\1


In [8]:
import pandas as pd
import os

# Path to your directory containing the CSV files
download_path = "C:/Users/supre/Documents/FAI/PROJECT/historical_data"
merged_data = pd.DataFrame()

# Loop through each file in the directory
for file in os.listdir(download_path):
    if file.endswith(".csv"):
        # Extract company name from the filename (without .csv extension)
        company_name = file.replace(".csv", "")
        
        # Load CSV into a DataFrame
        file_path = os.path.join(download_path, file)
        df = pd.read_csv(file_path)
        
        # Add a new column for the company name
        df["Company"] = company_name
        
        # Append to the merged DataFrame
        merged_data = pd.concat([merged_data, df], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_data.to_csv("C:/Users/supre/Documents/FAI/PROJECT/merged_stocks_data.csv", index=False)

print("Data merged successfully into merged_stocks_data.csv.")


Data merged successfully into merged_stocks_data.csv.


In [13]:
merged_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2019-05-31,43.181805,43.613062,42.877969,42.897572,108174400,0.0,0.0,AAPL
1,2019-06-03,43.027439,43.595909,41.721424,42.463867,161584400,0.0,0.0,AAPL
2,2019-06-04,42.988224,44.063910,42.762796,44.017353,123872000,0.0,0.0,AAPL
3,2019-06-05,45.154295,45.328268,44.384898,44.727940,119093600,0.0,0.0,AAPL
4,2019-06-06,44.860258,45.445882,44.632378,45.384624,90105200,0.0,0.0,AAPL


In [14]:
import numpy as np
import pandas as pd
import random
from collections import deque



In [16]:
# Load your merged data
data = pd.read_csv("C:/Users/supre/Documents/FAI/PROJECT/merged_stocks_data.csv")

# Filter data by company name (e.g., for a specific company)
company_data = data[data['Company'] == 'AAPL'].reset_index(drop=True)

# Q-learning parameters
actions = ["Buy", "Sell", "Hold"]  # Actions the agent can take
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995  # Decay rate for epsilon
q_table = {}  # Q-table for state-action pairs

# Initialize environment parameters
initial_balance = 1000  # Initial balance in dollars
shares_held = 0  # Initial shares held

# Function to get state
def get_state(data, t):
    return tuple(data.iloc[t][["Open", "High", "Low", "Close", "Volume"]])

# Reward function: calculates profit/loss for an action
def get_reward(action, current_price, shares_held, balance):
    if action == "Sell" and shares_held > 0:
        return shares_held * current_price
    elif action == "Buy" and balance >= current_price:
        return -current_price
    else:
        return 0  # "Hold" or not enough balance

# Q-learning algorithm
for episode in range(1, 101):  # Training for 100 episodes
    balance = initial_balance
    shares_held = 0
    total_reward = 0
    
    for t in range(len(company_data) - 1):
        state = get_state(company_data, t)
        
        # Initialize Q-values for unseen states
        if state not in q_table:
            q_table[state] = [0] * len(actions)
        
        # Epsilon-greedy action selection
        if random.uniform(0, 1) < epsilon:
            action_index = random.randint(0, len(actions) - 1)  # Explore
        else:
            action_index = np.argmax(q_table[state])  # Exploit
        
        action = actions[action_index]
        current_price = company_data.loc[t, "Close"]
        reward = get_reward(action, current_price, shares_held, balance)
        
        # Update balance and shares held based on action
        if action == "Buy" and balance >= current_price:
            balance -= current_price
            shares_held += 1
        elif action == "Sell" and shares_held > 0:
            balance += shares_held * current_price
            shares_held = 0
        
        total_reward += reward
        
        # Next state
        next_state = get_state(company_data, t + 1)
        
        # Q-learning update
        if next_state not in q_table:
            q_table[next_state] = [0] * len(actions)
        
        # Bellman equation
        old_q_value = q_table[state][action_index]
        next_max_q = max(q_table[next_state])
        q_table[state][action_index] = old_q_value + alpha * (reward + gamma * next_max_q - old_q_value)
    
    # Decay epsilon
    epsilon = max(epsilon * epsilon_decay, 0.01)
    # print(f"Episode {episode}, Total Reward: {total_reward}, Balance: {balance}")

print("Training complete.")

# Test phase (using the trained Q-table) can follow a similar approach, with the agent selecting the action with the highest Q-value for each state.


Training complete.


State and Q-Table: For each day in the test data, the agent retrieves the current state and checks for that state in the Q-table.

Exploitation (No Exploration): For each state in the test data, the agent selects the action with the highest Q-value.

Action Execution:
"Buy": Deduct the price of one stock from the balance and increase the shares_held.
"Sell": Add the value of all held shares to the balance and set shares_held to zero.

Portfolio Value Calculation: Tracks the total portfolio value (balance + value of held shares) to see the agent’s progress.

End of Testing: Prints the final portfolio value and total profit at the end of the test period